In [1]:
import pandas as pd
import logging
import dateutil
from dateutil import parser
import numpy as np
import matplotlib.pyplot as plt
import geopandas as gpd
from geopandas import GeoDataFrame
from shapely.geometry import Point
%matplotlib inline
pd.set_option('display.max_columns', 500)

In [2]:
#import dataset of project times from "data" folder
file = "../../../data/cleaned/all_quarters__one_record_per_project.csv"
output="../../Output/Entire Sample/"
df = pd.read_csv(file)

# Initial Data Cleaning

In [3]:
#First, filter out those projects that are exclusively non-residential (defined as those without units)
df = df[df['units'] > 0]

In [4]:
#Next, keep only those that reached completion at some point over the time period
#by leaving out this, we actually have less of a sample size for some of the charts 
#because there is more chance for duplicate building permits
#df=df[pd.notnull(df['comp_date'])]

In [5]:
df.shape

(3081, 29)

In [6]:
# drop those that don't have a firstfiled date.
df=df[pd.notnull(df['firstfiled'])]
df=df[df['firstfiled'] !='']

In [7]:
df.shape

(2793, 29)

In [8]:
#drop BP duplicates for now—this suggests that they are likely duplicates. 
#This includes records with BP "MULTIPLE" because these are mostly mega projects and we don't want them in the sample
df['bp_duplicates']=df.duplicated('dbi_permit', keep=False)
df=df[(df['bp_duplicates']==False) | (pd.isnull(df['dbi_permit']))]

In [9]:
df.shape

(2475, 30)

In [10]:
# make sure there are no duplicate first dates on same address-APN (aka drop those that have duplicate projects on same Address-APN combination)
df= df[~ df.duplicated(['address', 'apn', 'first_date'], keep=False)]

In [11]:
df.shape

(2475, 30)

In [12]:
df['project_time_years']=df['project_duration_days']/365

In [13]:
# Top 5 longest projects
df.sort_values('project_time_years', ascending = False)[0:5]

,Unnamed: 0,BP_date,address,aff,affnet,apn,best_date,best_stat,comp_date,con_date,dbi_permit,dropped_out,first_date,first_project_record_date,firstfiled,latest_project_record_date,latest_project_status,project_dates,project_duration_days,project_statuses,report_quarter,report_year,status,units,unitsnet,x,y,zoning,zoning_simplified,bp_duplicates,project_time_years
3353,0,NaN,833-881 JAMESTOWN,0.0,0.0,4991277,2007-09-17,CONSTRUCTION,04/01/2015,2007-09-17,9924080S,False,1991-02-19,2007-09-17,1991-02-19,2007-09-17,CONSTRUCTION,"('2007-09-17', '2007-09-17', '2007-09-17', '20...",8807.0,"('CONSTRUCTION', 'CONSTRUCTION', 'CONSTRUCTION...",1,2015,Under Construction,132.0,14.0,-122.392222,37.717174,RH-2,RH-2,False,24.128767
3021,0,2008-11-07,1179 TENNESSEE ST,0.0,0.0,4172053,2014-12-18,CONSTRUCTION,01/01/2015,2014-03-21,9902819,False,1999-02-11,2008-11-07,1999-02-11,2014-12-18,CONSTRUCTION,"('2008-11-07', '2009-11-03', '2009-11-03', '20...",5803.0,"('BP REINSTATED', 'BP REINSTATED', 'BP REINSTA...",4,2014,Under Construction,8.0,8.0,-122.389058,37.755993,UMU,UMU,False,15.898630
239,0,2014-02-12,480 POTRERO AV,11.0,11.0,3973002C,2016-01-13,CONSTRUCTION,07/01/2016,2015-03-26,201306250465,False,2002-03-22,2004-08-17,2002-03-22,2016-01-13,CONSTRUCTION,"('2004-08-17', '2012-10-04', '2012-10-04', '20...",5215.0,"('BP ISSUED', 'PL FILED', 'PL FILED', 'BP FILE...",2,2016,Under Construction,77.0,77.0,-122.407671,37.763478,UMU,UMU,False,14.287671
1523,0,NaN,950 CORBETT AV,0.0,0.0,2799013,2007-03-13,CONSTRUCTION,01/01/2016,2007-03-13,200201106632,False,2002-01-10,2007-03-13,2002-01-10,2007-03-13,CONSTRUCTION,"('2007-03-13', '2007-03-13', '2007-03-13', '20...",5104.0,"('CONSTRUCTION', 'CONSTRUCTION', 'CONSTRUCTION...",4,2015,Under Construction,3.0,3.0,-122.443730,37.751832,RM-1,RM-1,False,13.983562
3180,0,NaN,LUMINA (PLAZA C),0.0,0.0,3746001,2016-07-01,CONSTRUCTION,07/01/2016,2016-07-01,201207124725,False,2002-07-11,2016-07-01,2002-07-11,2016-07-01,CONSTRUCTION,"('2016-07-01',)",5104.0,"('CONSTRUCTION',)",2,2016,Under Construction,80.0,80.0,-122.391516,37.788413,RH DTR,RH DTR,False,13.983562


In [14]:
# Top 5 shortest projects
df.sort_values('project_time_years', ascending = True)[0:5]

,Unnamed: 0,BP_date,address,aff,affnet,apn,best_date,best_stat,comp_date,con_date,dbi_permit,dropped_out,first_date,first_project_record_date,firstfiled,latest_project_record_date,latest_project_status,project_dates,project_duration_days,project_statuses,report_quarter,report_year,status,units,unitsnet,x,y,zoning,zoning_simplified,bp_duplicates,project_time_years
1644,0,NaN,850 30TH AV,NaN,NaN,1672024,2010-02-24,CONSTRUCTION,04/01/2010,2010-02-24,200910269851,False,2009-10-26,2009-10-26,2009-10-26,2010-02-24,CONSTRUCTION,"('2009-10-26', '2010-02-24')",157.0,"('BP FILED', 'CONSTRUCTION')",1,2010,Under Construction,2.0,NaN,-122.489282,37.773244,RH-2,RH-2,False,0.430137
2046,0,NaN,414 SOMERSET ST,0.0,0.0,6044003,2015-10-05,CONSTRUCTION,2015-10-05,2015-10-05,201504103396,False,2015-04-10,2015-04-10,2015-04-10,2015-10-05,CONSTRUCTION,"('2015-04-10', '2015-10-05')",178.0,"('BP FILED', 'CONSTRUCTION')",3,2015,Under Construction,2.0,1.0,-122.407649,37.726252,RH-1,RH-1,False,0.487671
21,0,2012-10-26,55 05TH ST,NaN,NaN,3705039,2012-12-12,CONSTRUCTION,01/01/2013,2012-12-12,201206202986,False,2012-06-20,2012-06-20,2012-06-20 00:00:00,2012-12-12,CONSTRUCTION,"('2012-06-20', '2012-10-26', '2012-12-12')",195.0,"('BP FILED', 'BP APPROVED', 'CONSTRUCTION')",4,2012,Under Construction,116.0,8.0,-122.407185,37.783461,C-3-R,C-3-R,False,0.534247
1628,0,2012-10-23,28 PRECITA AV,NaN,NaN,5518023,2012-11-15,CONSTRUCTION,01/01/2013,2012-11-15,201206203050,False,2012-06-20,2012-06-20,2012-06-20 00:00:00,2012-11-15,CONSTRUCTION,"('2012-06-20', '2012-10-23', '2012-11-15')",195.0,"('BP FILED', 'BP ISSUED', 'CONSTRUCTION')",4,2012,Under Construction,2.0,1.0,-122.418489,37.746472,RH-2,RH-2,False,0.534247
1917,0,NaN,64 14TH ST,NaN,NaN,3529017,2009-08-07,CONSTRUCTION,10/01/2009,2009-08-07,200903194471,False,2009-03-19,2009-08-07,2009-03-19,2009-08-07,CONSTRUCTION,"('2009-08-07',)",196.0,"('CONSTRUCTION',)",3,2009,Under Construction,1.0,-1.0,-122.414942,37.768706,PDR-1-G,PDR-1-G,False,0.536986


In [15]:
#Manual Data Cleaning for some fishy values based on top 5 shortest list above
#df.loc[(df['address']=='55 05TH ST') & (new_df['apn']=='3705039'), 'units'] = 8 #looked up on PIM. Simple adding 8 units.
#df.loc[(df['address']=='545 POST ST') & (new_df['apn']=='0306022'), 'units'] = 4 #looked up on PIM. Simple adding 4 units
#df.loc[(df['address']=='555 POST ST') & (new_df['apn']=='0306020'), 'units'] = 17 #looked up on PIM. Simple adding 4 units
#df.loc[(df['address']=='515 JOHN MUIR DR') & (new_df['apn']=='7282005'), 'units'] = 2 #looked up on PIM. Change of use to residential. 2 additional units

#based on the above cases, unit count seems to be the unit count at the site at the end of 
#the project rather than the unit count of the project itself. Because of this, I will be adjust
#unit counts that are more than the net units added to the net units added amount. If units net
#is missing, just accept unit count as true for now.

def unit_change(value):
    return_val=value['units']
    if pd.notnull(value['unitsnet']):
        if value['unitsnet']<0:
            return_val = value['units']
        elif value['unitsnet']==0:
            return_val = value['units']
        elif value['unitsnet']<value['units']:
            return_val = value['unitsnet']
        else:
            return_val=value['units']
    return return_val

def aff_change(value):
    return_val=value['aff']
    if pd.notnull(value['affnet']):
        if value['affnet']<0:
            return_val = value['aff']
        elif value['affnet']==0:
            return_val = value['aff']
        elif value['affnet']<value['aff']:
            return_val = value['affnet']
        else:
            return_val=value['aff']
    return return_val

df['units']=df.apply(unit_change, axis=1)
df['aff']=df.apply(aff_change, axis=1)

In [16]:
df['project_time_years'].describe()

count    721.000000
mean       4.426379
std        3.052768
min        0.430137
25%        2.043836
50%        3.717808
75%        6.109589
max       24.128767
Name: project_time_years, dtype: float64

# Merge in Additional Affordable Housing Information

In [17]:
df.shape

(2475, 31)

In [18]:
#import data on LIHTC properties
lihtc=pd.read_excel("../../../data/Affordable-Projects/projects.xlsx", sheet_name='San Francisco')

In [19]:
lihtc[lihtc.duplicated('Project Address')==True]

,Application Number,Unnamed: 1,Type of tax credit funding,Project Name,Project Address,Project City,Project Zip Code,Project Phone Number,Project County,California Assembly District,California Senate District,Federal Congressional District,Census Tract,Assessor's Parcel Number (APN),Application Stage,Placed in Service (PIS) Date,Last Building PIS Date,Construction Type,Housing Type,Total Units,Low Income Units,Number of SRO/Studio Units,Number of 1 Bedroom Units,Number of 2 Bedroom Units,Number of 3 Bedroom Units,Number of 4 Bedroom Units,Number of 5 Bedroom Units,Number of 6 Bedroom Units,Units at or below 30% AMI,Units at 35% AMI,Units at 40% AMI,Units at 45% AMI,Units at 50% AMI,Units at 55% AMI,Units at 60% AMI,Owner or Applicant Name,Owner/Applicant Contact,Owner/Applicant Address,Owner/Applicant City,Owner/Applicant State,Owner/Applicant Zip,Owner/Applicant Phone,Owner/Applicant Fax,General Partner (1),General Partner (2),General Partner (3),General Partner (4),General Partner (5),Management Company,Management Company Contact,Management Company Address,Management Company City,Management Company State,Management Company Zip,Management Company Phone,Management Company Fax,Developer,Annual Federal Award,Total State Award
54,CA-2001-810,NaN,0.04,8th and Howard Family Apartments,"1166 Howard Street, San Francisco, CA 94103",San Francisco,94103,(415) 441-0260,San Francisco,17.0,11.0,12.0,178.02,3727-026,Placed In Service,2003-04-03,2003-04-03,New Construction,Large Family,74.0,73.0,0.0,12.0,40.0,22.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"1166 Howard Street Associates, a California L.P.",Brandon Flannery,215 Taylor Street,San Francisco,CA,94102,(415) 358-3935,(415) 409-8636,Howard Street Development Corporation,NaN,NaN,NaN,NaN,Tenderloin Neighborhood Development Corporation,Felicia Scruggs-Wright,215 Taylor Street,San Francisco,CA,94102,(415) 358-3903,(415) 409-8363,NaN,805553.0,0.0


In [20]:
#Take out just street address 
lihtc['address']=lihtc['Project Address'].str.split(",").str[0]
#capitalize all letters to match with dev pipeline addresses
lihtc['address']=lihtc['address'].str.upper()

In [21]:
#import and test fuzzy match
from fuzzywuzzy import fuzz
print (str(fuzz.partial_ratio("1166 Howard Street", "1166 Howard St.")))
print (str(fuzz.ratio("1166 Howard Street", "1166 Howard St.")))

93
85


In [22]:
#add variable for address of affordable development
def match_address(value):
    address = value['address']
    result=''
    for index, row in lihtc.iterrows():
        ratio = fuzz.partial_ratio(address, row['address'])
        if ratio>95:
            result= row['address']
        else:
            if result=='':
                result=''
            else:
                result=result
    return result

df['aff_address']=df.apply(match_address, axis=1)

    

In [23]:
#df[df['aff_address']!=''][['address', 'aff_address']]

In [24]:
#after manual inspection of matches above, throw out incorrect ones
df.loc[(df['address']=='2700 ARELIOUS WALKER DR'), 'aff_address'] = ''
df.loc[(df['address']=='2500 ARELIOUS WALKER DR'), 'aff_address'] = ''
df.loc[(df['address']=='2600 ARELIOUS WALKER DR'), 'aff_address'] = ''
df.loc[(df['address']=='1 FRANKLIN ST'), 'aff_address'] = ''

In [25]:
#keep only lihtc variables of interest before merge
lihtc=lihtc[['address', 'Low Income Units', 'Placed in Service (PIS) Date ']]

In [26]:
df=df.merge(lihtc, how='left', left_on='aff_address', right_on='address')

In [27]:
def aff_change2(value):
    return_val=value['aff']
    if value['aff_address']!='':
        if pd.isnull(value['aff']):
            return_val=value['Low Income Units']
        elif value['aff']==0:
            return_val=value['Low Income Units']
        else:
            return_val=value['aff']
    return return_val
            

df['aff']=df.apply(aff_change2, axis=1)

In [28]:
#visual inspection reveals that LIHTC projects are indeed probably dev pipeline projects. No further filtering needed
df[df['aff_address']!=''][['address_x', 'aff_address', 'aff', 'affnet', 'Low Income Units', 'first_date', 'Placed in Service (PIS) Date ']]

,address_x,aff_address,aff,affnet,Low Income Units,first_date,Placed in Service (PIS) Date
64,350 GOLDEN GATE AV,350 GOLDEN GATE AVENUE,69.0,NaN,69.0,2010-11-01,2012-08-01
391,227 WEST POINT ROAD,227 WEST POINT ROAD,158.0,158.0,106.0,2007-02-14,2012-12-14
426,2175 MARKET ST,2175 MARKET STREET,18.0,NaN,18.0,2006-08-31,2014-10-17
474,280 BEALE ST,280 BEALE STREET,69.0,0.0,69.0,2013-04-02,2015-06-23
614,850 BRODERICK ST,850 BRODERICK ST.,46.0,NaN,46.0,2006-12-20,2010-01-11
765,701 GOLDEN GATE AV,701 GOLDEN GATE AVENUE,99.0,99.0,99.0,2006-10-23,2012-11-30
885,365 FULTON STREET,365 FULTON STREET,120.0,120.0,120.0,2008-07-11,2011-07-15
1148,2800 ARELIOUS WALKER DR,2800 ARELIOUS WALKER DRIVE,29.0,0.0,29.0,2016-08-29,NaT
1258,150 OTIS ST,150 OTIS STREET,75.0,NaN,75.0,2010-05-14,2012-10-19
1345,125 MASON ST,125 MASON STREET,81.0,81.0,80.0,2006-06-30,2008-08-26


In [29]:
df.shape

(2475, 35)

## Merge in Inclusionary projects

In [30]:
iz=pd.read_csv('../../../data/Affordable-Projects/Residential_Projects_With_Inclusionary_Requirements.csv')

In [31]:
#Take out just street address 
iz['iz_address']=iz['Mapping Address'].str.split(",").str[0]
#capitalize all letters to match with dev pipeline addresses
iz['iz_address']=iz['iz_address'].str.upper()

In [32]:
#iz.head()

In [33]:
#add variable for address of affordable development
def match_address2(value):
    address = value['address_x']
    result=''
    for index, row in iz.iterrows():
        ratio = fuzz.partial_ratio(address, row['iz_address'])
        if ratio>95:
            result= row['iz_address']
        else:
            if result=='':
                result=''
            else:
                result=result
    return result

df['iz_address']=df.apply(match_address2, axis=1)

In [34]:
#df[df['iz_address']!=''][['address_x', 'iz_address']]


In [35]:
#after manual inspection of matches above, throw out incorrect ones
df.loc[(df['address_x']=='5670 MISSION ST'), 'iz_address'] = ''
df.loc[(df['address_x']=='231 FRANKLIN ST'), 'iz_address'] = ''
df.loc[(df['address_x']=='198 VALENCIA ST'), 'iz_address'] = ''
df.loc[(df['address_x']=='181 SOUTH PARK ST'), 'iz_address'] = ''
df.loc[(df['address_x']=='1169 MARKET ST'), 'iz_address'] = ''
df.loc[(df['address_x']=='1551 FRANKLIN ST'), 'iz_address'] = ''
df.loc[(df['address_x']=='2782 FOLSOM ST'), 'iz_address'] = ''
df.loc[(df['address_x']=='2909 BUSH ST'), 'iz_address'] = ''
df.loc[(df['address_x']=='1523 FRANKLIN ST'), 'iz_address'] = ''
df.loc[(df['address_x']=='424 OCTAVIA ST'), 'iz_address'] = ''
df.loc[(df['address_x']=='102-104 OCTAVIA STREET'), 'iz_address'] = ''
df.loc[(df['address_x']=='2001 17TH ST'), 'iz_address'] = ''
df.loc[(df['address_x']=='987 DOLORES ST'), 'iz_address'] = ''
df.loc[(df['address_x']=='1270 MISSION ST'), 'iz_address'] = ''
df.loc[(df['address_x']=='1225 DOLORES ST'), 'iz_address'] = ''
df.loc[(df['address_x']=='2855 FOLSOM ST'), 'iz_address'] = ''


In [36]:
iz=iz[['iz_address', 'Total BMRs in this Building or Phase', 'Planning Approval Date', 'Completion Date']]

In [37]:
#manually fix some duplicate IZs—assume earliest date and total units of multi-phase projects
iz.loc[(iz['iz_address']=='1717 17TH ST'), 'Planning Approval Date'] = '07/15/2010'
iz.loc[(iz['iz_address']=='1717 17TH ST'), 'Total BMRs in this Building or Phase'] = 2

iz.loc[(iz['iz_address']=='101 EXECUTIVE PARK BLVD'), 'Planning Approval Date'] = '06/16/2005'
iz.loc[(iz['iz_address']=='101 EXECUTIVE PARK BLVD'), 'Total BMRs in this Building or Phase'] = 18

iz.loc[(iz['iz_address']=='100 VAN NESS AVE'), 'Planning Approval Date'] = '02/07/2013'
iz.loc[(iz['iz_address']=='100 VAN NESS AVE'), 'Total BMRs in this Building or Phase'] = 50

iz.loc[(iz['iz_address']=='690 MARKET ST'), 'Planning Approval Date'] = '03/08/2004'
iz.loc[(iz['iz_address']=='690 MARKET ST'), 'Total BMRs in this Building or Phase'] = 0

iz.drop_duplicates('iz_address', inplace=True)

In [38]:
df.shape

(2475, 36)

In [39]:
df=df.merge(iz, how='left', left_on='iz_address', right_on='iz_address', indicator=True)

In [40]:
#as long as first-date and planning approval date from IZ data is not more than 20 years apart, assume they are the same project
import dateutil
from dateutil import parser


In [41]:
#finally, change affordable unit counts to that in the IZ data if the aff unit counts are blank in the dev pipeline data
def aff_change3(value):
    return_val=value['aff']
    if value['iz_address']!='':
        if (dateutil.parser.parse(value['first_date']) - dateutil.parser.parse(value['Planning Approval Date'])).days <= 7300:
            if pd.isnull(value['aff']):
                return_val=value['Total BMRs in this Building or Phase']
            elif value['aff']==0:
                return_val=value['Total BMRs in this Building or Phase']
            else:
                return_val=value['aff']
    return return_val
            

df['aff']=df.apply(aff_change3, axis=1)

In [42]:
df.shape

(2475, 40)

In [43]:
#export final set of cleaned, residential projects
df.to_csv('../../../data/cleaned/final_cleaned_projects.csv')